In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

## Описание набора данных и признаков

С сайта [avito.ru](https://www.avito.ru/) по состоянию на 12 августа 2020 года я спарсила 20716 объявлений о продаже автомобилей в Нижегородской области. Каждое объявление содержит технические характеристики автомобиля, а также цену продажи. 

Проект посвящен исследованию данных объявлений на предмет предсказания цены автомобиля на основании его технических характеристик.

In [2]:
COLUMNS = ['label', 'model', 'generation', 'modification', 'year', 'mileage', 'condition',
           'doors_num', 'body', 'engine', 'transmission', 'color', 'drive', 'wheel', 'package', 'price']

In [3]:
cars = pd.read_csv('data/avito_cars.csv', engine='python', names=COLUMNS)
cars.head()

,label,model,generation,modification,year,mileage,condition,doors_num,body,engine,transmission,color,drive,wheel,package,price
0,Audi,A4,B9 (2015—н. в.),35 TFSI 1.4 S tronic (150 л.с.),2017.0,89000 км,не битый,4.0,седан,бензин,робот,зелёный,передний,левый,NaN,1490000
1,Audi,Q5,I рестайлинг (2012—2017),2.0 TDI quattro S tronic (170 л.с.),2012.0,104000 км,не битый,5.0,внедорожник,дизель,робот,чёрный,полный,левый,Base,950000
2,Audi,A8,D4 рестайлинг (2013—2017),3.0 TFSI quattro Tiptronic (310 л.с.),2014.0,41000 км,битый,4.0,седан,бензин,автомат,чёрный,полный,левый,Base,960000
3,Audi,Q5,I (2008—2012),2.0 TFSI quattro S tronic (211 л.с.),2010.0,113043 км,не битый,5.0,внедорожник,бензин,робот,серый,полный,левый,Base,897000
4,Audi,Q7,4M (2015—н. в.),3.0 TDI quattro AT (249 л.с.),2017.0,149116 км,не битый,5.0,внедорожник,дизель,автомат,синий,полный,левый,Business,3700000


In [4]:
cars.shape

(20716, 16)

In [5]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20716 entries, 0 to 20715
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   label         20665 non-null  object 
 1   model         20664 non-null  object 
 2   generation    20664 non-null  object 
 3   modification  20664 non-null  object 
 4   year          20664 non-null  float64
 5   mileage       20171 non-null  object 
 6   condition     20171 non-null  object 
 7   doors_num     20664 non-null  float64
 8   body          20664 non-null  object 
 9   engine        20664 non-null  object 
 10  transmission  20664 non-null  object 
 11  color         20664 non-null  object 
 12  drive         20664 non-null  object 
 13  wheel         20645 non-null  object 
 14  package       17386 non-null  object 
 15  price         20715 non-null  object 
dtypes: float64(2), object(14)
memory usage: 2.5+ MB


Пройдёмся по переменным:
- label - марка автомобиля
- model - модель
- generation - поколение
- modification  - модификация
- year  - год выпуска автомобиля
- mileage  - пробег 
- condition  - состояние автомобиля (бинарный признак)
- doors_num - количество дверей
- body - тип кузова
- engine - тип двигателя
- transmission - коробка передач
- color - цвет автомобиля
- drive - тип привода
- wheel - руль
- package - комплектация
- price - цена, целевая переменная

## Предобработка данных (Data Preprocessing)

В первую очередь требуется удалить 51 строку, в которых значения всех переменных, кроме цены, пропущены. Возможно, при обработке данных объявлений парсером произошла какая-то ошибка.

In [6]:
cars[cars['model'].isna()].head()

,label,model,generation,modification,year,mileage,condition,doors_num,body,engine,transmission,color,drive,wheel,package,price
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1295000
89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,490000
389,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,270000
463,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,560000
818,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000000


In [7]:
# Удаляем строки
cars = cars[~cars['model'].isna()]

### 1. Обработка дупликатов
Посмотрим, содержатся ли в датасете повторяющиеся объявления, и удалим одинаковые записи:

In [8]:
print("Before:", cars.shape[0])

Before: 20664


In [9]:
cars.duplicated().value_counts()

False    20489
True       175
dtype: int64

Видим, что в датасете 175 дупликата. Избавимся от них: 

In [10]:
cars.drop_duplicates(inplace=True)
print("After:", cars.shape[0])

After: 20489


### 2. Обработка пропущенных значений
Теперь займёмся пропусками:

In [11]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20489 entries, 0 to 20715
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   label         20489 non-null  object 
 1   model         20489 non-null  object 
 2   generation    20489 non-null  object 
 3   modification  20489 non-null  object 
 4   year          20489 non-null  float64
 5   mileage       20000 non-null  object 
 6   condition     20000 non-null  object 
 7   doors_num     20489 non-null  float64
 8   body          20489 non-null  object 
 9   engine        20489 non-null  object 
 10  transmission  20489 non-null  object 
 11  color         20489 non-null  object 
 12  drive         20489 non-null  object 
 13  wheel         20470 non-null  object 
 14  package       17239 non-null  object 
 15  price         20489 non-null  object 
dtypes: float64(2), object(14)
memory usage: 2.7+ MB


Имеем следующие признаки с пропущенными значениями:
- mileage (пробег)
- condition (состояние)
- wheel (руль)
- package (комплектация) 

In [12]:
# Преобразуем тип признака из object в int
cars['mileage'] = cars[~cars['mileage'].isna()]['mileage'].apply(lambda x: x.split()[0]).astype('int')

# Заменим пропущенные значения средним значением пробега
cars['mileage'] = cars['mileage'].fillna(int(cars['mileage'].mean()))

In [13]:
# Заменяем пропущенные значения модой
cars['condition'] = cars['condition'].fillna(cars['condition'].mode()[0])
cars['wheel'] = cars['wheel'].fillna(cars['wheel'].mode()[0])
cars['package'] = cars['package'].fillna(cars['package'].mode()[0])

In [14]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20489 entries, 0 to 20715
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   label         20489 non-null  object 
 1   model         20489 non-null  object 
 2   generation    20489 non-null  object 
 3   modification  20489 non-null  object 
 4   year          20489 non-null  float64
 5   mileage       20489 non-null  float64
 6   condition     20489 non-null  object 
 7   doors_num     20489 non-null  float64
 8   body          20489 non-null  object 
 9   engine        20489 non-null  object 
 10  transmission  20489 non-null  object 
 11  color         20489 non-null  object 
 12  drive         20489 non-null  object 
 13  wheel         20489 non-null  object 
 14  package       20489 non-null  object 
 15  price         20489 non-null  object 
dtypes: float64(3), object(13)
memory usage: 2.7+ MB


### 3. Преобразование переменных 

In [15]:
cars['price'] = cars['price'].apply(lambda s: ''.join([x for x in s if x.isdigit()])).astype('int32')

Посмотрим на признак **generation**

In [16]:
cars['generation'].value_counts()

I (2004—2013)                  581
I (1977—н. в.)                 376
I (2011—2018)                  375
I рестайлинг (2003—2010)       374
I (2007—2013)                  334
                              ... 
I (1992—1995)                    1
III (1996—2002)                  1
I (1994—2006)                    1
C216 рестайлинг (2010—2014)      1
GD (2001—2008)                   1
Name: generation, Length: 1200, dtype: int64

1200 уникальных значений - слишком много. К тому же, основное различие в признаках составляют года, указанные в скобках. Эта информация избыточна, поэтому нужно преобразовать признак.  

Идея: добавим бинарный признак *restyling*

In [17]:
print("Cars with restyling:", sum(cars['generation'].apply(lambda x: 'рестайлинг' in x)))
cars['restyling'] = cars['generation'].apply(lambda x: 'рестайлинг' in x).map({True: 'Да', False: 'Нет'})

Cars with restyling: 5960


In [18]:
cars['generation'] = cars['generation'].apply(lambda x: x.replace('рестайлинг ', ''))
cars['generation'].value_counts()

I (2004—2013)         581
I (2011—2018)         381
I (1977—н. в.)        376
I (2003—2010)         374
I (2007—2013)         334
                     ... 
X100 (1998—2001)        1
W463 3 (2015—2017)      1
S10 (1982—1990)         1
VIII (1995—2000)        1
GD (2001—2008)          1
Name: generation, Length: 1094, dtype: int64

Уникальных значений стало 1094. Теперь отбросим информацию в скобках и оставим только порядковый номер поколения.

In [19]:
cars['generation'] = cars['generation'].apply(lambda x: x.split()[0])
cars['generation'].value_counts()

I        10283
II        2716
III       1380
IV         534
V          290
         ...  
Y33          1
R172         1
F45          1
T26          1
GF/GC        1
Name: generation, Length: 334, dtype: int64

Рассмотрим признак **modification**

In [20]:
cars['modification'].value_counts()

1.6 MT (81 л.с.)                  487
1.6 MT (98 л.с.)                  481
1.6 MT (87 л.с.)                  408
1.7 MT (80 л.с.)                  350
1.6 MT (106 л.с.)                 291
                                 ... 
2.0 D MT (90 л.с.)                  1
2.5 TURBO 4WD MT (265 л.с.)         1
S 350 3.5 4MATIC AT (272 л.с.)      1
1.6 AT (135 л.с.)                   1
190 E 2.0 MT (105 л.с.)             1
Name: modification, Length: 2264, dtype: int64

Имеем 2264 уникальных значений. Заметим, что в модификации содержится избыточная информация о коробке передач (MT - механическая, AT - автомат). Оставим в этом признаке только число лошадиных сил и преобразуем признак из категориального в вещественный.

In [21]:
cars['modification'] = cars['modification'].apply(lambda x: x[x.find('(')+1:-1].split()[0]).astype('int')

In [22]:
cars.head()

,label,model,generation,modification,year,mileage,condition,doors_num,body,engine,transmission,color,drive,wheel,package,price,restyling
0,Audi,A4,B9,150,2017.0,89000.0,не битый,4.0,седан,бензин,робот,зелёный,передний,левый,Базовая,1490000,Нет
1,Audi,Q5,I,170,2012.0,104000.0,не битый,5.0,внедорожник,дизель,робот,чёрный,полный,левый,Base,950000,Да
2,Audi,A8,D4,310,2014.0,41000.0,битый,4.0,седан,бензин,автомат,чёрный,полный,левый,Base,960000,Да
3,Audi,Q5,I,211,2010.0,113043.0,не битый,5.0,внедорожник,бензин,робот,серый,полный,левый,Base,897000,Нет
4,Audi,Q7,4M,249,2017.0,149116.0,не битый,5.0,внедорожник,дизель,автомат,синий,полный,левый,Business,3700000,Нет


Посмотрим на признак **package**:

In [23]:
cars['package'].value_counts()[:20]

Базовая       11386
Comfort         983
Base            576
Luxe            353
Standard        277
Classic         262
Elegance        236
Prestige        217
Active          182
SE              182
Стандарт        166
Люкс            146
Cosmo           135
Sport           132
Норма           125
Ghia            121
Norma           114
Premium         110
Expression      100
Titanium        100
Name: package, dtype: int64

Видно, что многие уникальные значения означают одно и то же, только записаны по-разному (Norma и Норма, например). Заменим некоторые значения:

In [24]:
packages = {
    'Базовая': 'Base', 'Base': 'Base',
    'SE': 'SE', 'Особая серия': 'SE',
    'Lux': 'Luxe', 'Люкс': 'Luxe',
    'Норма': 'Norma', 'Norma': 'Norma',
    'Sport': 'Sport', 'HSE': 'HSE',
    'Стандарт': 'Standard', 'Standart': 'Standard',             
    'Комфорт': 'Comfort', 'Confort': 'Comfort',
    'Comfort': 'Comfort', 'Premium': 'Premium',
    'Enjoy': 'Enjoy', 'Executive': 'Executive',
    'Special edition': 'SE', 'Limited Edition': 'LE',
    'Limited': 'LE', 'Active': 'Active',
    'Prestige': 'Prestige', 'Invite': 'Invite',
    'Business': 'Business', 'Trendline': 'Trend'
}

In [25]:
def same_package(value):
    for package in packages:
        if package in value:
            return packages.get(package)
    return value

In [26]:
cars['package'] = cars['package'].apply(lambda x: same_package(x))

In [27]:
cars['package'].value_counts()

Base            11966
Comfort          1396
Luxe              772
Standard          491
SE                411
                ...  
In13B               1
Impulse Line        1
21109 Консул        1
Shogun              1
Momentum+           1
Name: package, Length: 430, dtype: int64

### 4. Выявление аномалий 

In [28]:
cars.describe()

,modification,year,mileage,doors_num,price
count,20489.000000,20489.000000,20489.000000,20489.000000,2.048900e+04
mean,123.404070,2008.163063,151018.297818,4.393919,4.839746e+05
std,49.879753,7.478546,87838.121650,0.745142,5.740985e+05
min,23.000000,1937.000000,1.000000,2.000000,1.000000e+00
25%,89.000000,2005.000000,95000.000000,4.000000,1.700000e+05
50%,110.000000,2009.000000,140000.000000,5.000000,3.400000e+05
75%,143.000000,2013.000000,190000.000000,5.000000,5.900000e+05
max,585.000000,2020.000000,1000000.000000,5.000000,1.372340e+07


Из таблицы видно, что в датасете присутствует аномальное объявление с ценой "1".   
Также максимальное значение пробега 1.000.000 км выглядит подозрительно, как и пробег 1 км, нужно удалить строки с данными значениями.

In [38]:
cars = cars[(cars['price'] < cars.price.quantile(0.99)) & (cars['price'] > cars.price.quantile(0.01))]
cars = cars[~((cars['mileage'] <= 1000) | (cars['mileage'] >= 900000))]

In [40]:
cars.describe()

,modification,year,mileage,doors_num,price
count,19983.000000,19983.000000,19983.000000,19983.000000,1.998300e+04
mean,122.566982,2008.168243,151567.233649,4.393535,4.499874e+05
std,47.640082,7.307467,84961.586753,0.745293,4.149588e+05
min,23.000000,1937.000000,1145.000000,2.000000,2.800000e+04
25%,89.000000,2006.000000,95472.500000,4.000000,1.750000e+05
50%,110.000000,2009.000000,141000.000000,5.000000,3.399900e+05
75%,143.000000,2013.000000,190000.000000,5.000000,5.800000e+05
max,585.000000,2020.000000,807000.000000,5.000000,2.899000e+06


Сохраним очищенные и подготовленные данные в файл preprocessed_data.csv

In [44]:
cars.to_csv('data/preprocessed_data.csv', index=False)